In [14]:
# %pip install sklearn

In [15]:
import numpy as np
import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
# from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [16]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

# df_rC4_1_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
# df_rC4_5_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')

df_rC4_1_75 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
df_rC4_5_75 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')




In [17]:
print(df_rC4_1_75.info())
print(df_rC4_5_75.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47882 entries, 0 to 47881
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                47882 non-null  object 
 1   425_pct_Al          47882 non-null  float64
 2   C4_pct_Eth          47882 non-null  float64
 3   C4_pct_H2O          47882 non-null  float64
 4   HydWtr_pct_Ammonia  47882 non-null  float64
 5   HydWtr_Na2O         47882 non-null  float64
 6   Butanol             47882 non-null  float64
 7   Decanol             47882 non-null  float64
 8   DI55102             47882 non-null  float64
 9   DI55152             47882 non-null  float64
 10  FC55003             47882 non-null  float64
 11  FC55009             47882 non-null  float64
 12  FC55552             47882 non-null  float64
 13  FC55569             47882 non-null  float64
 14  FC55576             47882 non-null  float64
 15  FFC55555            47882 non-null  float64
 16  LC55

In [18]:
print(df_rC4_1_75.head())
print(df_rC4_5_75.head())

                  Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790    2.520640    21.98715   

   HydWtr_pct_Ammonia  HydWtr_Na2O    Butanol   Decanol   DI55102   DI55152  \
0            0.908765     2.712180  54.858300  0.722511  0.970496  0.925255   
1            0.907643     2.805415  54.858300  0.722511  0.970496  0.925255   
2            0.905959     2.945270  54.858300  0.722511  0.970496  0.925255   
3            0.904837     3.038505  51.190050  0.733904  0.970653  0.923773   
4            0.903715     3.131740  48.744567  0.741499  0.970307  0.924103   

   ...    LC55557    LC90366    LC90368   PI55004   PI55020  TC55552  \
0  ...  65.873400  61.011100  51.234400  1.306310 -0.607993  177.188   
1  ...  

# 1 75

In [19]:
# Calculate the differences and create new columns
df_rC4_1_75['FFC55555_diff'] = df_rC4_1_75['FFC55555'] - df_rC4_1_75['FFC55555_adj']
df_rC4_1_75['Butanol_diff'] = df_rC4_1_75['Butanol'] - df_rC4_1_75['Butanol_adj']
df_rC4_1_75['Decanol_diff'] = df_rC4_1_75['Decanol'] - df_rC4_1_75['Decanol_adj']


df_rC4_1_75['rC4_diff'] = (df_rC4_1_75['FFC55555']*df_rC4_1_75['FC55552'])-(df_rC4_1_75['FFC55555_adj']*df_rC4_1_75['FC55552'])

## ^^^^^^ That's the number to solve for ^^^^^^^

In [20]:

df_rC4_1_75['Date'] = pd.to_datetime(df_rC4_1_75['Date'])
df_rC4_1_75.set_index('Date', inplace=True)


In [21]:
# Sum up the hourly data to get daily totals for each component
daily_totals_1_75 = df_rC4_1_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_1_75 = df_rC4_1_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].groupby(df_rC4_1_75.index.year).resample('D').sum()


In [22]:
# Initialize a dictionary to store the annual production values for each year
annual_production_1_75 = {}

# Loop through each group (each year) in the daily totals
for year, group in daily_totals_by_year_1_75.groupby(level=0):  # level=0 refers to the 'year' group
    # Calculate the average daily totals for the year
    average_daily_totals_1_75 = group.mean()
    
    # Count the actual days with data in that year
    actual_days_with_data_1_75 = group.dropna().shape[0]
    
    # Calculate the estimated annual production based on actual days with data
    estimated_annual_production_FFC55555_1_75 = average_daily_totals_1_75['FFC55555_diff'] * actual_days_with_data_1_75
    estimated_annual_production_butanol_1_75 = average_daily_totals_1_75['Butanol_diff'] * actual_days_with_data_1_75
    estimated_annual_production_decanol_1_75 = average_daily_totals_1_75['Decanol_diff'] * actual_days_with_data_1_75

    estimated_annual_production_rC4_1_75 = average_daily_totals_1_75['rC4_diff'] * actual_days_with_data_1_75
    
    # Store results in dictionary
    annual_production_1_75[year] = {
        'FFC55555_diff': estimated_annual_production_FFC55555_1_75,
        'Butanol_diff': estimated_annual_production_butanol_1_75,
        'Decanol_diff': estimated_annual_production_decanol_1_75,
        'rC4_diff': estimated_annual_production_rC4_1_75
    }

# Output the results
for year in annual_production_1_75:
    print(f"Year {year}:")
    print(f"  Estimated Annual Production for FFC55555_diff: {annual_production_1_75[year]['FFC55555_diff']}")
    print(f"  Estimated Annual Production for Butanol_diff: {annual_production_1_75[year]['Butanol_diff']}")
    print(f"  Estimated Annual Production for Decanol_diff: {annual_production_1_75[year]['Decanol_diff']}")
    print(f"  Estimated Annual Production for rC4_diff: {annual_production_1_75[year]['rC4_diff']}")


Year 2012:
  Estimated Annual Production for FFC55555_diff: 11.310000000000011
  Estimated Annual Production for Butanol_diff: -972.1568279333324
  Estimated Annual Production for Decanol_diff: -193.90538045999924
  Estimated Annual Production for rC4_diff: 378578.3373750003
Year 2013:
  Estimated Annual Production for FFC55555_diff: 13.170000000000009
  Estimated Annual Production for Butanol_diff: -570.8263937000012
  Estimated Annual Production for Decanol_diff: -68.90767924999939
  Estimated Annual Production for rC4_diff: 491051.11237500043
Year 2014:
  Estimated Annual Production for FFC55555_diff: 5.250000000000004
  Estimated Annual Production for Butanol_diff: -468.30548340000047
  Estimated Annual Production for Decanol_diff: -117.12876973099984
  Estimated Annual Production for rC4_diff: 192716.19787500016
Year 2015:
  Estimated Annual Production for FFC55555_diff: 10.650000000000006
  Estimated Annual Production for Butanol_diff: -72.39918870002506
  Estimated Annual Produc

# 5 75

In [23]:
# Calculate the differences and create new columns
df_rC4_5_75['FFC55555_diff'] = df_rC4_5_75['FFC55555'] - df_rC4_5_75['FFC55555_adj']
df_rC4_5_75['Butanol_diff'] = df_rC4_5_75['Butanol'] - df_rC4_5_75['Butanol_adj']
df_rC4_5_75['Decanol_diff'] = df_rC4_5_75['Decanol'] - df_rC4_5_75['Decanol_adj']

df_rC4_5_75['rC4_diff'] = (df_rC4_5_75['FFC55555']*df_rC4_5_75['FC55552'])-(df_rC4_5_75['FFC55555_adj']*df_rC4_5_75['FC55552'])

In [24]:
df_rC4_5_75['Date'] = pd.to_datetime(df_rC4_5_75['Date'])
df_rC4_5_75.set_index('Date', inplace=True)



In [25]:
# Sum up the hourly data to get daily totals for each component
daily_totals = df_rC4_5_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_5_75 = df_rC4_5_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].groupby(df_rC4_5_75.index.year).resample('D').sum()


In [26]:
# Initialize a dictionary to store the annual production values for each year
annual_production_5_75 = {}

# Loop through each group (each year) in the daily totals
for year, group in daily_totals_by_year_5_75.groupby(level=0):  # level=0 refers to the 'year' group
    # Calculate the average daily totals for the year
    average_daily_totals_5_75 = group.mean()
    
    # Count the actual days with data in that year
    actual_days_with_data_5_75 = group.dropna().shape[0]
    
    # Calculate the estimated annual production based on actual days with data
    estimated_annual_production_FFC55555_5_75 = average_daily_totals_5_75['FFC55555_diff'] * actual_days_with_data_5_75
    estimated_annual_production_butanol_5_75 = average_daily_totals_5_75['Butanol_diff'] * actual_days_with_data_5_75
    estimated_annual_production_decanol_5_75 = average_daily_totals_5_75['Decanol_diff'] * actual_days_with_data_5_75

    estimated_annual_production_rC4_5_75 = average_daily_totals_5_75['rC4_diff'] * actual_days_with_data_5_75
    
    # Store results in dictionary
    annual_production_5_75[year] = {
        'FFC55555_diff': estimated_annual_production_FFC55555_5_75,
        'Butanol_diff': estimated_annual_production_butanol_5_75,
        'Decanol_diff': estimated_annual_production_decanol_5_75,
        'rC4_diff': estimated_annual_production_rC4_5_75
    }

# Output the results
for year in annual_production_5_75:
    print(f"Year {year}:")
    print(f"  Estimated Annual Production for FFC55555_diff: {annual_production_5_75[year]['FFC55555_diff']}")
    print(f"  Estimated Annual Production for Butanol_diff: {annual_production_5_75[year]['Butanol_diff']}")
    print(f"  Estimated Annual Production for Decanol_diff: {annual_production_5_75[year]['Decanol_diff']}")
    print(f"  Estimated Annual Production for rC4_diff: {annual_production_5_75[year]['rC4_diff']}")



# Assuming the 'Date' column is a string, you can extract the year using the str accessor
# annual_production_5_75['Year'] = annual_production_5_75['Date'].str[:4]

# Alternatively, if the 'Date' column is already a datetime object, you can extract the year directly
annual_production_5_75['Year'] = annual_production_5_75['Date'].dt.year

# Now count the records for each year
record_counts_by_year = annual_production_5_75['Year'].value_counts().sort_index()

# Output the results
print(record_counts_by_year)


Year 2012:
  Estimated Annual Production for FFC55555_diff: 68.38000000000012
  Estimated Annual Production for Butanol_diff: -3498.0317270333326
  Estimated Annual Production for Decanol_diff: -152.43169980999917
  Estimated Annual Production for rC4_diff: 2380522.0060000024
Year 2013:
  Estimated Annual Production for FFC55555_diff: 132.17000000000016
  Estimated Annual Production for Butanol_diff: -3723.1329483000004
  Estimated Annual Production for Decanol_diff: 43.63221600000063
  Estimated Annual Production for rC4_diff: 4745483.903250002
Year 2014:
  Estimated Annual Production for FFC55555_diff: 143.3200000000002
  Estimated Annual Production for Butanol_diff: -8778.798418299997
  Estimated Annual Production for Decanol_diff: 58.367250789000195
  Estimated Annual Production for rC4_diff: 5372815.018000004
Year 2015:
  Estimated Annual Production for FFC55555_diff: 130.87000000000023
  Estimated Annual Production for Butanol_diff: -8454.699985400028
  Estimated Annual Productio

KeyError: 'Date'

In [ ]:
# Initialize a dictionary to store the annual production values for each year
annual_production = {}

# Loop through each group (each year) in the daily totals
for year, group in daily_totals_by_year.groupby(level=0):  # level=0 refers to the 'year' group
    # Calculate the average daily totals for the year
    average_daily_totals = group.mean()
    
    # Count the actual days with data in that year
    actual_days_with_data = group.dropna().shape[0]
    
    # Calculate the estimated annual production based on actual days with data
    estimated_annual_production_butanol = average_daily_totals['Butanol_diff'] * actual_days_with_data
    estimated_annual_production_decanol = average_daily_totals['Decanol_diff'] * actual_days_with_data
    
    # Store results in dictionary
    annual_production[year] = {
        'Butanol_diff': estimated_annual_production_butanol,
        'Decanol_diff': estimated_annual_production_decanol
    }

# Output the results
for year in annual_production:
    print(f"Year {year}:")
    print(f"  Estimated Annual Production for Butanol_diff: {annual_production[year]['Butanol_diff']}")
    print(f"  Estimated Annual Production for Decanol_diff: {annual_production[year]['Decanol_diff']}")


In [ ]:
print(df_rC4_1_75['FFC55555'].describe())
print(df_rC4_5_75['FFC55555'].describe())

print(df_rC4_1_75['FFC55555_adj'].describe())
print(df_rC4_5_75['FFC55555_adj'].describe())